In [1]:
import pandas as pd
import numpy as np

In [2]:
df=pd.read_csv('C:\devdev\sentiments\IMDB Dataset.csv')

In [3]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [6]:
df.duplicated().sum()

418

In [7]:
df['sentiment'].value_counts()

sentiment
positive    25000
negative    25000
Name: count, dtype: int64

In [8]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

In [9]:
df.drop_duplicates()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative


In [10]:
df['review']= df['review'].str.lower()

In [11]:
#remove html tags using regex
import re
def remove_html_tags(text):
    pattern= re.compile('<.*?>')
    return pattern.sub(r'', text)

In [12]:
df['review']=df['review'].apply(remove_html_tags)

In [13]:
df.head()

,review,sentiment
0,one of the other reviewers has mentioned that ...,positive
1,a wonderful little production. the filming tec...,positive
2,i thought this was a wonderful way to spend ti...,positive
3,basically there's a family where a little boy ...,negative
4,"petter mattei's ""love in the time of money"" is...",positive


In [15]:
import string
def remove_punc1(text):
    if isinstance(text, str):  # Process only strings
            return text.translate(str.maketrans('', '', string.punctuation)).strip()
    return text 

In [16]:
df['review']= df['review'].astype(str).apply(remove_punc1)

In [18]:
import spacy
nlp = spacy.load("en_core_web_sm")

def remove_stopwords(text):
    if pd.isna(text):  # nan 
        return ""
    doc = nlp(text)
    filtered_tokens = [token.text for token in doc if not token.is_stop]
    return " ".join(filtered_tokens)

df["reviews_cleaner"] = df["review"].apply(remove_stopwords)


In [31]:
df= df.drop(columns='review')

In [32]:
df

,sentiment,reviews_cleaner
0,positive,reviewers mentioned watching 1 oz episode ll h...
1,positive,wonderful little production filming technique ...
2,positive,thought wonderful way spend time hot summer we...
3,negative,basically s family little boy jake thinks s zo...
4,positive,petter matteis love time money visually stunni...
...,...,...
49995,positive,thought movie right good job nt creative origi...
49996,negative,bad plot bad dialogue bad acting idiotic direc...
49997,negative,catholic taught parochial elementary schools n...
49998,negative,m going disagree previous comment maltin secon...


In [134]:
X= df.drop(columns='sentiment')
y= df['sentiment']

In [115]:
print(X.shape)
print(y.shape)

(50000, 1)
(50000,)


In [59]:
X

,reviews_cleaner
0,reviewers mentioned watching 1 oz episode ll h...
1,wonderful little production filming technique ...
2,thought wonderful way spend time hot summer we...
3,basically s family little boy jake thinks s zo...
4,petter matteis love time money visually stunni...
...,...
49995,thought movie right good job nt creative origi...
49996,bad plot bad dialogue bad acting idiotic direc...
49997,catholic taught parochial elementary schools n...
49998,m going disagree previous comment maltin secon...


In [135]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()

y= encoder.fit_transform(y)

In [136]:
y.shape

(50000,)

In [137]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test= train_test_split(X,y, test_size=0.2, random_state=1)

In [138]:
X_train.shape

(40000, 1)

bag of words

In [139]:
from sklearn.feature_extraction.text import CountVectorizer
vec= CountVectorizer()

In [140]:
vec = CountVectorizer(max_features=5000)  
X_train_bow = vec.fit_transform(X_train['reviews_cleaner']).toarray()
X_test_bow = vec.transform(X_test['reviews_cleaner']).toarray()

In [130]:
X_test_bow= X_test_bow.reshape(-1,1)

In [131]:
X_train_bow= X_train_bow.reshape(-1,1)

In [141]:
print(X_train_bow.shape)
print(X_test_bow.shape)

(40000, 5000)
(10000, 5000)


In [142]:
y_train.shape

(40000,)

Navie bayess

In [143]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()

gnb.fit(X_train_bow,y_train)

GaussianNB()

In [145]:
y_pred=gnb.predict(X_test_bow)
from sklearn.metrics import accuracy_score,confusion_matrix
accuracy_score(y_test,y_pred)

0.7623

In [148]:
confusion_matrix(y_test,y_pred)

array([[4346,  698],
       [1679, 3277]], dtype=int64)

random forest classifier

In [169]:
from sklearn.ensemble import RandomForestClassifier
classify= RandomForestClassifier()

classify.fit(X_train_bow, y_train)

RandomForestClassifier()

In [170]:
y_preds_classify= classify.predict(X_test_bow)
from sklearn.metrics import accuracy_score,confusion_matrix
accuracy_score(y_test,y_preds_classify)

0.8415

tf-idf

In [172]:
X_train

,reviews_cleaner
18165,film compared hilarious british comedy fish ca...
36059,reasonably effective horrorsciencefiction la a...
13242,inspiration naked gun movies casts leslie niel...
32985,film originally released promoted notably unim...
41133,happened chance friends house started watching...
...,...
43723,largerthanlife figures wyatt earp bat masterso...
32511,okay havepenelope keith miss herringbonetweed ...
5192,odd willfully skewed biopic dyan thomas hear l...
12172,basic structure story beginning middle endsome...


In [173]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf= TfidfVectorizer(max_features=5000)

X_train_tf= tfidf.fit_transform(X_train['reviews_cleaner'])
X_test_tf= tfidf.fit(X_test['reviews_cleaner'])